In [2]:
import pandas as pd
import numpy as np

In [4]:
nyc = pd.read_csv('C:/Users/Александр/Projects/projects/NYC_dataset.csv')

In [5]:
# написать функцию haversine для подсчета расстояния между пунктами отправления и прибытия и,
# используя  pd.iterrows(), создать колонку distance с подсчитанным расстоянием.
# φ1 и φ2 — широта исходного местоположения и широта пункта назначения в радианах, 
# λ1 и λ2 — долгота исходного местоположения и долгота пункта назначения в радианах,
# r — радиус сферы, будем считать его равным 6371.

In [6]:
nyc.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [7]:
# f1 - pickup_latitude
# f2 - dropoff_latitude
# l1 - pickup_longitude
# l2 - dropoff_longitude
def haversine(l1, f1, l2, f2, radius=6371.):
    f1, f2, l1, l2 = map(np.radians, [f1, f2, l1, l2])
    diff_lon = l2 - l1 
    diff_lat = f2 - f1
    
    a = np.sin(diff_lat/2.0)**2 + np.cos(f1) * np.cos(f2) * np.sin(diff_lon/2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return radius * c

In [8]:
%%time
nyc['distance'] = pd.Series(haversine(row.pickup_longitude,
                                      row.pickup_latitude,
                                      row.dropoff_longitude,
                                      row.dropoff_latitude) for i, row in nyc.iterrows())

CPU times: total: 3.23 s
Wall time: 3.22 s


In [9]:
# Давайте попробуем еще один метод, pd.itertuples, который на каждой итерации возвращает 
# именованный кортеж (named tuple) для каждой строки датафрейма. 
# Параметр index по умолчанию равен True, поэтому первым элементом кортежа следует индекс, далее — содержимое строки. 

# Обратите внимание, что, в отличие от первого метода, строка возвращается уже не как Series:

for row in nyc[:1].itertuples(name='ride'):  # исп. "ride" как название
    print(f"Тип содержимого: {type(row)}")

Тип содержимого: <class 'pandas.core.frame.ride'>


In [10]:
# Посмотрим на формат вывода:

for row in nyc[:3].itertuples(name='custom_name'):
    print(row)
    print()

custom_name(Index=0, key='2009-06-15 17:26:21.0000001', fare_amount=4.5, pickup_datetime='2009-06-15 17:26:21 UTC', pickup_longitude=-73.844311, pickup_latitude=40.721319, dropoff_longitude=-73.84161, dropoff_latitude=40.712278000000005, passenger_count=1, distance=1.030763935048818)

custom_name(Index=1, key='2010-01-05 16:52:16.0000002', fare_amount=16.9, pickup_datetime='2010-01-05 16:52:16 UTC', pickup_longitude=-74.016048, pickup_latitude=40.711303, dropoff_longitude=-73.979268, dropoff_latitude=40.782004, passenger_count=1, distance=8.450133595805992)

custom_name(Index=2, key='2011-08-18 00:35:00.00000049', fare_amount=5.7, pickup_datetime='2011-08-18 00:35:00 UTC', pickup_longitude=-73.982738, pickup_latitude=40.76127, dropoff_longitude=-73.991242, dropoff_latitude=40.750562, passenger_count=2, distance=1.3895252257697308)



In [11]:
# Индекс и число пассажиров для первых трех наблюдений:

for row in nyc[:3].itertuples():
    print("Индекс {}; Число пассажиров: {}".format(row.Index, row.passenger_count))

Индекс 0; Число пассажиров: 1
Индекс 1; Число пассажиров: 1
Индекс 2; Число пассажиров: 2


In [12]:
# Задача: проделайте ту же операцию, что и в предыдущем шаге, но уже используя pd.itertuples(), 
# а затем сравните время выполнения.

In [13]:
def haversine(l1, f1, l2, f2, radius=6371.):
    f1, f2, l1, l2 = map(np.radians, [f1, f2, l1, l2])
    diff_lon = l2 - l1 
    diff_lat = f2 - f1
    
    a = np.sin(diff_lat/2.0)**2 + np.cos(f1) * np.cos(f2) * np.sin(diff_lon/2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return radius * c

In [14]:
%%time
nyc['distance'] = pd.Series(haversine(x.pickup_longitude,
                                      x.pickup_latitude,
                                      x.dropoff_longitude,
                                      x.dropoff_latitude) for x in nyc.itertuples())

CPU times: total: 594 ms
Wall time: 602 ms


In [15]:
# Еще один часто используемый метод pd.apply() применяет заданную функцию вдоль определенной
# оси (axis=0 – строки; axis=1 – столбцы), и является более эффективным способом, чем pd.iterrows() 
# за счет использования внутренних оптимизаций в pandas. Тем не менее, все равно подразумевает итерацию по всем рядам.

In [16]:
%%time
nyc['distance'] = nyc[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']] \
                 .apply(lambda x: haversine(x[0], x[1], x[2], x[3]), axis=1)

CPU times: total: 1.06 s
Wall time: 1.07 s


In [17]:
%%time
nyc['distance'] = nyc.apply(lambda x: haversine(x.pickup_longitude,
                                                x.pickup_latitude, 
                                                x.dropoff_longitude, 
                                                x.dropoff_latitude), axis=1)

CPU times: total: 1.81 s
Wall time: 1.82 s


In [18]:
%%time
nyc['distance'] = nyc.apply(lambda x: haversine(x['pickup_longitude'], 
                                                x['pickup_latitude'],
                                                x['dropoff_longitude'], 
                                                x['dropoff_latitude']), axis=1)

CPU times: total: 1.31 s
Wall time: 1.32 s


In [19]:
# В переменную longest сохраните самое большое расстояние

In [20]:
longest = nyc.distance.max()
longest

8667.818812217454

In [21]:
# В целом, itertuples — это именно то, что нам нужно! Но есть пара моментов.
# Оказывается, в нашем случае itertuples — не предел. На всех предыдущих шагах мы использовали циклы, что не очень хорошо.
# Почему же?
# Базовые объекты в pandas — Series и DataFrame, которые представляют собой одномерные и двумерные массивы (arrays) 
# соответственно. Так, векторизация — процесс выполнения операций над массивами. 
# В pandas есть довольно большой набор векторизованных функций, и их использование во многих случаях предпочтительнее 
# обычных циклов, поскольку они предназначены для работы именно с Series. К таким функциям относятся, например, sum и agg. 
# С помощью векторизации можно получить ещё большее ускорение, значительно уменьшив количество итераций. 
# Иными словами, действия будут происходят сразу над всей серией, а не над каждым элементом по отдельности.

# Изменения в коде весьма незначительны: передаем функции на вход необходимые колонки, и получается магия!

In [22]:
# df['distance'] = haversine(df['col1'], df['col2'], df['col3'], df['col4'])

In [23]:
# Убедимся, что векторизованные операции работают намного быстрее. 
# Создайте функцию для подсчета расстояния haversine, описанную в первом шаге, и примените её к датасету nyc. 
# Результат сохраните в колонку distance.  

In [25]:
%%time
nyc['distance'] = haversine(nyc.pickup_longitude,
                            nyc.pickup_latitude,
                            nyc.dropoff_longitude,
                            nyc.dropoff_latitude)

CPU times: total: 0 ns
Wall time: 6.98 ms
